# Rand(Bool) is fast but sum(Rand(Bool, 10)) is not...

## Full for-loop

In [70]:
function coin_experiment1()
    ffst = Int32[]
    frand = Int32[]
    fmin = Int32[]
    for i in 1:100000
        crand = rand(1:1000)
        tmin = 10
        for j in 1:1000
            tfreq = 0
            for k in 1:10
                tfreq += rand(Bool)
            end
            
            if j == 1
                push!(ffst, tfreq)
            end       
            if j == crand
                push!(frand, tfreq)
            end
            
            if tfreq < tmin
                tmin = tfreq
            end
        end
        push!(fmin, tmin)
    end
    return [mean(ffst)/10, mean(frand)/10, mean(fmin)/10]
end

coin_experiment1 (generic function with 1 method)

In [71]:
@time coin_experiment1()

  4.847132 seconds (10.90 k allocations: 3.484 MB)


3-element Array{Float64,1}:
 0.50041 
 0.500474
 0.037935

## Creating 1D array with rand()

In [10]:
function coin_experiment2()
    ffst = Int32[]
    frand = Int32[]
    fmin = Int32[]
    for i in 1:100000
        crand = rand(1:1000)
        tmin = 10
        for j in 1:1000
#             tfreq = 0
#             for k in 1:10
#                 tfreq += rand(Bool)
#             end
            tfreq = sum(rand(Bool,(10,1)))
            if j == 1
                push!(ffst, tfreq)
            end
            
            if j == crand
                push!(frand, tfreq)
            end
            
            if tfreq < tmin
                tmin = tfreq
            end
        end
        push!(fmin, tmin)
    end
    return [mean(ffst)/10, mean(frand)/10, mean(fmin)/10]
end

coin_experiment2 (generic function with 1 method)

In [11]:
@time coin_experiment2()

 24.641817 seconds (200.01 M allocations: 10.434 GB, 10.18% gc time)


3-element Array{Float64,1}:
 0.499869
 0.499632
 0.037475

## Creating 2D array with rand()

In [15]:
function coin_experiment3()
    ffst = Int32[]
    frand = Int32[]
    fmin = Int32[]
    for i in 1:100000
        coin10x1000_fliped = sum(rand(Bool, (1000,10)), 2)
        crand = rand(1:1000)
        minval, cmin = findmin(coin10x1000_fliped)
        push!(ffst, coin10x1000_fliped[1])
        push!(frand, coin10x1000_fliped[crand])
        push!(fmin, coin10x1000_fliped[cmin])
    end
    return [mean(ffst)/10, mean(frand)/10, mean(fmin)/10]
end

coin_experiment3 (generic function with 2 methods)

In [16]:
@time coin_experiment3()

  6.718427 seconds (1.31 M allocations: 1.731 GB, 2.56% gc time)


3-element Array{Float64,1}:
 0.500858
 0.49965 
 0.037425

# The cause is rand(Bool,10) or sum()?

In [24]:
function rand_experiment1()
    for i in 1:100_000_000
        rand(Bool,10)
    end
    return
end

rand_experiment1 (generic function with 1 method)

In [25]:
@time rand_experiment1()

 15.080562 seconds (200.00 M allocations: 8.941 GB, 8.83% gc time)


In [27]:
function rand_experiment2()
    for i in 1:1_000_000_000 # 10 times of experiment1
        rand(Bool)
    end
    return
end

rand_experiment2 (generic function with 1 method)

In [28]:
@time rand_experiment2()

  3.938521 seconds (2.35 k allocations: 118.228 KB)


In [37]:
function rand_experiment3()
    for i in 1:100_000_000 # same as experiment1
        [rand(Bool), rand(Bool), rand(Bool), rand(Bool), rand(Bool), rand(Bool), rand(Bool), rand(Bool), rand(Bool), rand(Bool)]
    end
    return
end

rand_experiment3 (generic function with 1 method)

In [38]:
@time rand_experiment3()

 14.145010 seconds (100.01 M allocations: 7.451 GB, 7.79% gc time)


In [42]:
function rand_experiment4() # no rand()
    for i in 1:100_000_000 # same as experiment1
        [i for i in 1:10] # list comprehension is not fast in julia
    end
    return
end

rand_experiment4 (generic function with 1 method)

In [41]:
@time rand_experiment4()

 14.288016 seconds (100.01 M allocations: 13.411 GB, 14.38% gc time)


# Conclusion

Julia seems to have a problem in creating array efficiently.